In [9]:
#Data loading and preprocessing
import pandas as pd 
import numpy as np

In [19]:
#Load dataset
dataset = pd.read_csv("datasets/SNR_20.csv")
dataset.head(5)

,Unnamed: 0,received_signal_real_0,received_signal_real_1,received_signal_real_2,received_signal_real_3,received_signal_real_4,received_signal_real_5,received_signal_real_6,received_signal_real_7,received_signal_imaginary_0,received_signal_imaginary_1,received_signal_imaginary_2,received_signal_imaginary_3,received_signal_imaginary_4,received_signal_imaginary_5,received_signal_imaginary_6,received_signal_imaginary_7,channel,transmitted_bits_0,transmitted_bits_1,transmitted_bits_2,transmitted_bits_3,transmitted_bits_4,transmitted_bits_5,transmitted_bits_6,transmitted_bits_7,transmitted_bits_8,transmitted_bits_9,transmitted_bits_10,transmitted_bits_11,optimal_OFDM_BER
0,0,3.001762,-4.580632,2.490680,-0.578107,2.795580,-1.503200,3.125641,1.421548,5.196654,-2.844841,0.960775,-1.255957,4.985869,-0.693276,1.417358,3.244947,[1. +0.j 0. +0.j 0.3+0.3j],0,0,0,0,0,1,0,1,0,1,0,1,0.000000
1,1,2.669055,3.011873,3.014480,0.970410,2.816612,4.944164,3.240250,0.826746,4.403751,-4.754855,1.046819,2.987016,5.115613,3.037655,1.242330,2.520570,[1. +0.j 0. +0.j 0.3+0.3j],1,0,0,0,1,0,1,0,1,0,1,0,0.083333
2,2,2.632133,2.911608,2.514619,-1.924853,3.504592,1.186973,2.789156,1.725545,5.145146,-4.356428,1.235730,-1.279845,4.943174,0.938635,1.034576,2.925029,[1. +0.j 0. +0.j 0.3+0.3j],1,0,0,0,0,0,0,1,1,1,1,1,0.083333
3,3,2.529888,4.006828,2.768672,-0.599116,3.388057,4.398308,2.675267,0.847009,4.847335,0.422788,1.025481,2.525339,4.813047,3.146778,1.103249,2.906121,[1. +0.j 0. +0.j 0.3+0.3j],1,0,1,1,1,1,1,0,1,0,1,0,0.083333
4,4,3.039367,1.142946,3.196247,1.477741,3.657999,4.352656,2.696841,0.835797,4.705504,-1.235550,1.483852,-0.339612,4.811335,3.058956,1.180234,2.873091,[1. +0.j 0. +0.j 0.3+0.3j],1,1,0,1,1,1,0,1,1,0,1,0,0.000000


In [20]:
#Load features (X) and targets (Y) from the dataset
#X represents real and imaginary parts of the received IQ symbols
#Y represents the transmitted binary stream we would like to recover (12 bits in this example)

X = dataset.iloc[:,1:17]
X = X.values
Y_bits = dataset.iloc[:,18:30]
Y = Y_bits.values

print("Dimensions of X:", X.shape)
print("Dimensions of Y:", Y.shape)
print(Y)

Dimensions of X: (500000, 16)
Dimensions of Y: (500000, 12)
[[0 0 0 ... 1 0 1]
 [1 0 0 ... 0 1 0]
 [1 0 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 1 ... 1 0 1]
 [1 0 0 ... 1 1 0]]


**Dataset Pre-processing**

In [21]:
#Import the standard scaler from sklearn preprocessing tools
from sklearn.preprocessing import StandardScaler

#Scale the IQ samples using the standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

[[-0.27294355 -1.24514648 -0.71433129 ... -0.23236226 -1.20995895
  -0.94407788]
 [-0.41708298  0.80596071 -0.46508701 ...  0.95792137 -1.29220764
  -1.36892649]
 [-0.43307855  0.77887419 -0.70294016 ...  0.2882682  -1.38983476
  -1.13171064]
 ...
 [-0.12041531  0.50561184 -0.564127   ... -0.46442125  0.30988987
   0.66585048]
 [-0.35014158  1.0725886  -0.49550381 ... -0.1936418   0.31819774
   0.80657128]
 [-0.17684876  0.7082978  -0.51265087 ...  0.00515121  0.15481505
   0.78732333]]


In [22]:
from sklearn.model_selection import train_test_split

#Reading Question/Answer responses for decoder/encoder
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

print("Number of training samples:", len(X_train))
print("Number of test samples", len(X_test))

Number of training samples: 450000
Number of test samples 50000


**Model Building**

In [23]:
#Import modules for neural network building and training
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.utils import plot_model

In [24]:
#Define Neural Network Architecture
def model_builder():
  input  = Input(shape=(16,), name="model_input")
  dense1 = Dense(50, activation='relu')(input)
  dropout1 = Dropout(0.35)(dense1)
  dense2 = Dense(50, activation='relu')(dropout1)
  dropout2 = Dropout(0.35)(dense2)
  dense3 = Dense(50, activation='relu')(dropout2)
  dropout3 = Dropout(0.35)(dense3)
  dense4 = Dense(50, activation='relu')(dropout3)
  output1 = Dense(12, activation = 'sigmoid', name="Output_bits")(dense4)

  model = Model([input], [output1])

  return model

In [25]:
#Build and compile model
model = model_builder()
model.compile(
    optimizer="sgd",
    loss={
        "Output_bits": 'mean_squared_error',
        },
    metrics=["mse"]
)

**Model Training**

In [26]:
#Train the model
model.fit(
    {"model_input": X_train},
    {"Output_bits": y_train},
    epochs=50,
    batch_size=20,
    validation_split=0.1
)

Epoch 1/50
20250/20250 [==============================] - 25s 1ms/step - loss: 0.2495 - mse: 0.2495 - val_loss: 0.2278 - val_mse: 0.2278
Epoch 2/50
20250/20250 [==============================] - 25s 1ms/step - loss: 0.2307 - mse: 0.2307 - val_loss: 0.1986 - val_mse: 0.1986
Epoch 3/50
20250/20250 [==============================] - 27s 1ms/step - loss: 0.2131 - mse: 0.2131 - val_loss: 0.1782 - val_mse: 0.1782
Epoch 4/50
20250/20250 [==============================] - 24s 1ms/step - loss: 0.2015 - mse: 0.2015 - val_loss: 0.1631 - val_mse: 0.1631
Epoch 5/50
20250/20250 [==============================] - 24s 1ms/step - loss: 0.1930 - mse: 0.1930 - val_loss: 0.1507 - val_mse: 0.1507
Epoch 6/50
20250/20250 [==============================] - 25s 1ms/step - loss: 0.1863 - mse: 0.1863 - val_loss: 0.1404 - val_mse: 0.1404
Epoch 7/50
20250/20250 [==============================] - 25s 1ms/step - loss: 0.1802 - mse: 0.1802 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 8/50
20250/20250 [=================

**Predictions on the test set**

In [29]:
#Predict on test set
predictions = model.predict(X_test)

#Map predictions to 0s and 1s
for i in range(0,len(X_test)):
  for j in range(0,12):
    if predictions[i][j] > 0.5:
      predictions[i][j] = 1
    else:
      predictions[i][j] = 0

**Evaluation in terms of Bit-Error-Rate (BER)**

In [30]:
#Compute BER on test set predictions
errors = 0
BER = 0
for i in range(0,len(X_test)):
  for j in range(0,12):
    if predictions[i][j] != y_test[i][j]:
      errors+=1
  BER = BER + errors/12
  errors = 0
BER = BER/len(X_test)

print("Model BER:", BER)
print("Conventional OFDM BER:", sum(dataset.optimal_OFDM_BER)/len(dataset.optimal_OFDM_BER))

Model BER: 0.06293666666667244
Conventional OFDM BER: 0.11163216666634748
